# How do I _copy_ a file from the _Public Reference Files_ repository?
## Overview
Files are stored on the Platform in two spaces: Public Reference Files and projects of which you're a member. Files can be copied from these two repositories.

## Objective
Here we focus on **copying a file from the Public Reference Files repository**. [[reference](http://docs.sevenbridges.com/docs/copy-a-file)]

## Procedure
1. We'll list our projects to make sure our target project exists.
2. Then, we'll list the files within the Public Reference Files repository.
3. Lastly, we'll copy a file from the source project to our target project.

## Prerequisites
1. You need to be a member (or owner) of at least one project. Learn more about creating a new project. [[recipe](projects_makeNew.ipynb)] [[reference](http://docs.sevenbridges.com/docs/create-a-new-project)].
 2. You need your **authentication token** and the API needs to know about it. See <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a> for details. Learn more about [obtaining your authentication token](http://docs.sevenbridges.com/v1.0/docs/get-your-authentication-token).
 3. You understand how to list projects of which you are a member. We will use this call directly to select a project below. [[recipe](projects_listAll.ipynb)] [[reference](http://docs.sevenbridges.com/docs/list-all-your-projects)]
 5. You understand how to list files within a project. We will use this call below and select a file from the Public Reference Files repository. [[recipe](files_listAll.ipynb)] [[reference](http://docs.sevenbridges.com/docs/list-files-primary-method)]

Note that as with all okAPI recipes, this recipe makes use of the [Seven Bridges Public API Python library](../Tutorials/SBPLAT/Setup_API_environment.ipynb).
 
## Imports
We import the `Api` class from the official `sevenbridges-python` bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The `Api` object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbpla, etc}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Copy a file from the Public Reference
First, we will check that our target project, `my\_project` exists. Then, we will specify our `source\_project`, the Public Reference Files repository. Then, we will iterate through this project to find two specific files<sup>1</sup> and copy a file from `source\_project` to `my\_project`. Lastly, we will list the files in the target project to make sure the fiels have been successfully copied.

The critical information for this POST is the **`file_id`**. Note, you are allow to copy the same file as many times as you like. However, duplicates will be automatically have a prefix attached of (\_1\_, \_2\_, etc) depending on how many times you copy the file.

To make these results very obvious, use an empty project as your `my\_project` (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or change the `name` in the `data` dictionary to something like 'Dept of Awesome.copy'.

<sup>1</sup> Files are only accessible **within** a project - here the Public Reference project (**warning** we may change this project name in the future).

In [ ]:
# [USER INPUT] Set your project name; source project ID; and files to copy here:
my_project_name = 'Keep on Smiling'
source_project_id = 'admin/sbg-public-data'        
files_list = ['WES_human_Illumina.pe_1.fastq',
             'WES_human_Illumina.pe_2.fastq'
             ]


# Double-check that your project exists
my_project = [p for p in api.projects.query(limit=100).all()
                  if p.name == my_project_name]
if not my_project:
    print('Your project (%s) not found, check spelling' % my_project_name)
    raise KeyboardInterrupt
else:
    my_project = my_project[0]
    
# LIST all file names in your project and file objects from source project
my_file_names = [f.name for f in 
    api.files.query(limit = 100, project = my_project).all()
]

source_files = api.files.query(
    limit = 100, project = source_project_id, names = files_list
)

if not source_files:
    print('Source files not found, check spelling and project id')
    raise KeyboardInterrupt
    
# Copy files if they don't already exist in my_project
for f in source_files:
    if f.name in my_file_names:
        print('File already exists in second project, skipping')
    else:
        print('File {} does not exist in {}; copying now'.format(
              f.name, my_project.name))

        new_file = f.copy(project = my_project, name = f.name)

        # re-list files in target project to verify the copy worked
        my_files = [f.name for f in api.files.query(
                limit = 100,project = my_project).all()]

        if f.name in my_files:
            print('Sucessfully copied one file!')
        else:
            print('Something went wrong...')

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/copy-a-file).